In [2]:
from IPython.display import Code

# Example: Validate Model Outputs for correctness

For safety-critical applications we can not accept deviations at the model ouputs caused by the deployment method. The following shows how to verify if the generated model outputs are as expected.

*Warning:* The current version can only verify the bit-excactness of model outputs. Hence why it is very sensitive to even small derivations compared to the reference (golden) outputs. This limitation might be eliminated with a future revision of MLonMCUs `validate` feature.

## Supported components

**Models:** Any (`aww` and `toycar` used below)

**Frontends:** TFLite only (`tflite`)

**Frameworks/Backends:** Any (`tvmaotplus` and `tflmi` used below)

**Platforms/Targets:** `tvm`, `microtvm` and `mlif`

**Features:** `validate_new` (aka. `gen_data`, `gen_ref_data`, `set_inputs` & `get_outputs`) and `gen_ref_labels` features have to be enabled 

## Prerequisites

Set up MLonmCU as usual, i.e. initialize an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

In [3]:
Code(filename="environment.yml.j2")

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "{{ home_dir }}"
logging:
  level: INFO
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 5
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: results
  # Directory where custom extensions can be integrated
  plugins: plugins
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "{{ home_dir }}/models"
# Here default clone_urls
repos:
  tensorflow:
    url: "https://github.com/tensorflow/tflite-micro.git"
    ref: 93d4b1ccc6e0d1a9062acb1d79f21505daf9d940
  tvm:
    url: "https://github.com/apache/tvm.git"
    ref: a9fcac1a47f4b57c1d9d828c7ed1d77452b02cc5
    options:
      recursive: true
  etiss:
    url: "https://github.com/tum-ei-eda/etiss.git"
    ref: 40c0b920cf4072adf189211ca44291534355dcfd
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: 99624237110838fb564f2e6fce838f1962744635
  tflite_pack:
    url: "https://github.com/tum-ei-eda/tflite-pack.git"
    ref: 2d31cafc941d3889b042af9c0a72b5f768ac4606
frameworks:
  default: tvm
  tflm:
    enabled: true
    backends:
      default: tflmi
      tflmi:
        enabled: true
        features: {}
  tvm:
    enabled: true
    backends:
      default: tvmaotplus
      tvmaotplus:
        enabled: true
        features: {}
      tvmllvm:
        enabled: true
        features: {}
    features: {}
frontends:
  tflite:
    enabled: true
    features:
      split_layers: true
toolchains:
  gcc: true
platforms:
  mlif:
    enabled: true
    features:
      debug: true
      validate: true
targets:
  default: host_x86
  etiss:
    enabled: true
    features: {}
postprocesses:
  use: []
vars:
  allow_extensions: false
  runs_per_stage: true
  riscv_gcc.dl_url: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/GCC/default/2023.11.27/Ubuntu/20.04/rv32gc_ilp32d.tar.xz"
flags: {}

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

## Usage

*Hint*: Due to the program being build in debug mode and running one inference for each provided input-output combination, the simulation time will likely decrease by some factors. Add the `--parallel` flag to your command line to allow MLonMCU to run multiple simulations in parallel.

*Hint:* We are not able to provide reference data for every model in out model zoo. If you might want to add reference data for your own models, see: TODO

### A) Command Line Interface

As an example, let's see if the `tflmi` and `tvmaotplus` backend produce different model outputs for the same model.

In [11]:
!python3 -m mlonmcu.cli.main flow run aww --target host_x86 --backend tvmaotplus --backend tflmi \
    -f validate_new --post validate_outputs -c gen_data.number=10 \
    -c validate_labels.validate_metrics="allclose(atol=0.1,rtol=0.1);topk(n=1)" \
    -c mlif.template_version=v2 -c set_inputs.interface=stdin_raw -c get_outputs.interface=stdout_raw

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - [session-735]  Processing stage LOAD
INFO - [session-735]  Processing stage BUILD
INFO - [session-735]  Processing stage COMPILE
INFO - [session-735]  Processing stage RUN
INFO - [session-735]  Processing stage POSTPROCESS
INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-735] Done processing runs
INFO - Report:
   Session  Run Model Frontend Framework     Backend Platform    Target  Runtime [s]  Validation  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data                                           Features                                             Config       Postprocesses Comment     topk(n=1)     topk(n=2)
0      735    0   aww   tflite       tvm  tvmaotplus     mlif  host_x86     0.024452        True      90161      56856          53560     34085      2516        40               56816  [gen_ref_data, set_inputs, va

In [12]:
!python3 -m mlonmcu.cli.main flow run aww --target host_x86 --backend tvmaotplus --backend tflmi \
    -f validate_new -f gen_ref_labels --post validate_labels -c gen_data.number=10 \
    -c validate_labels.classify_metrics="topk_label(n=1)" \
    -c mlif.template_version=v2 -c set_inputs.interface=stdin_raw -c get_outputs.interface=stdout_raw

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - [session-736]  Processing stage LOAD
INFO - [session-736]  Processing stage BUILD
INFO - [session-736]  Processing stage COMPILE
INFO - [session-736]  Processing stage RUN
INFO - [session-736]  Processing stage POSTPROCESS
INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-736] Done processing runs
INFO - Report:
   Session  Run Model Frontend Framework     Backend Platform    Target  Runtime [s]  Validation  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data                                           Features                                             Config      Postprocesses Comment topk_label(n=1)
0      736    0   aww   tflite       tvm  tvmaotplus     mlif  host_x86     0.025166        True      90161      56856          53560     34085      2516        40               56816  [gen_ref_labels, gen_data, gen_ref_data, s